# Python and REST APIs

Python can be used instead of `curl` for accessing REST APIs. The most useful library for this is [requests](https://requests.readthedocs.io/en/latest/). When combined with the `json` library in Python, we can easily write small programs, and wrap them into command-line utilities.

We simply import the `requests` library, and use it to retrieve data. The OpenAlex API returns json data, and the request library makes it easy to access that in the form of a Python dictionary. Here is a short example.



In [3]:
import requests

req = requests.get('https://api.openalex.org/institutions?search=carnegie+mellon+university')
data = req.json()
data



{'meta': {'count': 4, 'db_response_time_ms': 25, 'page': 1, 'per_page': 25},
 'results': [{'id': 'https://openalex.org/I74973139',
   'ror': 'https://ror.org/05x2bcf33',
   'display_name': 'Carnegie Mellon University',
   'relevance_score': 298677.62,
   'country_code': 'US',
   'type': 'education',
   'homepage_url': 'http://www.cmu.edu/index.shtml',
   'image_url': 'https://upload.wikimedia.org/wikipedia/commons/1/1d/Www.wikipedia.org_screenshot_2018.png',
   'image_thumbnail_url': 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/1d/Www.wikipedia.org_screenshot_2018.png/76px-Www.wikipedia.org_screenshot_2018.png',
   'display_name_acronyms': ['CMU'],
   'display_name_alternatives': [],
   'works_count': 107978,
   'cited_by_count': 9708044,
   'ids': {'openalex': 'https://openalex.org/I74973139',
    'ror': 'https://ror.org/05x2bcf33',
    'mag': '74973139',
    'grid': 'grid.147455.6',
    'wikipedia': 'https://en.wikipedia.org/wiki/Web%20server%20directory%20index',
    'wik

In [15]:
data.keys()

dict_keys(['meta', 'results', 'group_by'])

In [6]:
data['meta']['count']

4

In [10]:
[result['display_name'] for result in data['results']]

['Carnegie Mellon University',
 'Carnegie Mellon University Qatar',
 'Carnegie Mellon University Africa',
 'Carnegie Mellon University Australia']

In [12]:
names = []
for i in range(data['meta']['count']):
    names += [data['results'][i]['display_name']]
names    

['Carnegie Mellon University',
 'Carnegie Mellon University Qatar',
 'Carnegie Mellon University Africa',
 'Carnegie Mellon University Australia']

Now, it is easy to replicate the example we had from last class to show each result with the works_count and cited_by_count, even with simple formatting.



In [13]:
for result in data['results']:
    print(f'{result["display_name"]:50s}{result["works_count"]:10d}{result["cited_by_count"]:10d}')



Carnegie Mellon University                            107978   9708044
Carnegie Mellon University Qatar                         549     11205
Carnegie Mellon University Africa                         87       468
Carnegie Mellon University Australia                      27       188


## Our first Python based shell script



It is convenient to use the notebook for this, but let's convert this to a script that takes an argument. Let's do this in a few pieces.

1. create a file called oa_inst.py, and make it executable.

In the file, add these lines:

```
#!/usr/bin/env python

import sys
print(sys.argv)
```

The first line (the so-called shebang line) tells the shell what interpreter to use, in this case, that it is a python script. Then we import the `sys` module. This module provides basic access to command line arguments through the .argv attribute.

Run your script with a few examples:

    ./oa_inst.py
    ./oa_inst.py carnegie mellon university
    
You can see the first element of `sys.argv` is always the script name. All the other elements are what we call the command-line arguments. 



We need to join these with + as we did in the shell script before. It is easy to do in Python. Now, add these lines:

```
query = '+'.join(sys.argv[1:])
url = f'https://api.openalex.org/institutions?search={query}'

import requests

req = requests.get(url)
data = req.json()

for result in data['results']:
    print(f'{result["display_name"]:50s}{result["works_count"]:10d}{result["cited_by_count"]:10d}')
```

Now you should be able to run this python script like the shell script.

You can move the script to ~/bin if you put that on your path like we described in the first lecture, and then run that from anywhere.

Our script is not without issues. They aren't big issues, but we can *only* use this script in the shell. We can't import it and use it here in the notebook. If you do import it, you will see that it tries to run something, but something weird happens, and it doesn't work right.

We need to separate some things out so we can have a script *and* importable library.



In [16]:
import oa_inst

query = -f+/home/jupyter-jkitchin@andrew.cm-11dd7/.local/share/jupyter/runtime/kernel-9d4b67b6-ec08-42d4-bb3b-83f2ac3b8e83.json
status code: 200
No results found


In [17]:
oa_inst.req

<Response [200]>

## Getting better than sys.argv

`sys.argv` is really only suitable for the simplest of command line arguments, and it isn't really even great for those. Among the limitations are:

1. No option parsing (or you have to write your own)
2. No built-in help or documentation

Some built-in core libraries in Python can help with this, e.g. [argparse](https://docs.python.org/3/library/argparse.html). There are also third-party libraries like [click](https://palletsprojects.com/p/click/). 

Let's rewrite the script above using click. The principle idea is we write a function that does what we want with some arguments, use the click library to convert the command line arguments into variables we use in the function, and then, we only run the function when we run the script (as opposed to importing from it).

Making this look easy requires some advanced Python skills. Let's work out a reusable function first. I am writing this with some 20/20 hindsight we don't have yet. We will return later to why we wrote the function this specific way. This function should take a list of terms, or a string to query. Either way, we convert it to a string with each word joined by +. Then, we return a formatted string for each result found.



In [18]:
import requests 
from collections.abc import Iterable 

def openalex_institution(query):
    'query is a list of terms in the query, or a string.'
    
    # Replace spaces with +
    if isinstance(query, str):
        query = '+'.join(query.split())

    # If it is not a string We assume it is an iterable of strings.
    elif isinstance(query, Iterable):
        query = '+'.join(query)
        
    else:
        raise Exception('query should be a string or Iterable')
        
    url = f'https://api.openalex.org/institutions?search={query}'
    req = requests.get(url)
    data = req.json()

    return [f'{result["display_name"]:50s}{result["works_count"]:10d}{result["cited_by_count"]:10d}'
            for result in data['results']]
            
openalex_institution('carnegie mellon university')            

['Carnegie Mellon University                            107978   9708044',
 'Carnegie Mellon University Qatar                         549     11205',
 'Carnegie Mellon University Africa                         87       468',
 'Carnegie Mellon University Australia                      27       188']

In [19]:
# Test with list of words
openalex_institution(['carnegie', 'mellon', 'university'])          

['Carnegie Mellon University                            107978   9708044',
 'Carnegie Mellon University Qatar                         549     11205',
 'Carnegie Mellon University Africa                         87       468',
 'Carnegie Mellon University Australia                      27       188']

Note our function returns data in the form of a list of strings. Later, we can join them into a single string like this.



In [20]:
print('\n'.join(openalex_institution(['carnegie', 'mellon', 'university'])))



Carnegie Mellon University                            107978   9708044
Carnegie Mellon University Qatar                         549     11205
Carnegie Mellon University Africa                         87       468
Carnegie Mellon University Australia                      27       188


## Basic click usage
That is the independent reusable part. Now, let's look at how click works. We have to create a function that does what we want, and then decorate it with click functions. Start by creating a new file: oa_inst2.py with these contents. The `main` function is what will run in our script, and only when we run this as a script.

```
import click

@click.command(help='OpenAlex Institutions')
@click.argument('query', nargs=-1)
def main(query):
    print(query)
    
if __name__ == '__main__':
    main()
```

Now, you can see we automatically get help



In [21]:
! ./oa_inst2.py --help



Usage: oa_inst2.py [OPTIONS] [QUERY]...

  OpenAlex Institutions

Options:
  --help  Show this message and exit.


We can also run the command with a few arguments. Here you see that the arguments become a tuple of strings. 



In [22]:
! ./oa_inst2.py carnegie mellon university



('carnegie', 'mellon', 'university')


Now, we combine the function we worked out above in the script. We make a few modifications to the main function here. First, we have to join the strings returned by the openalex_institution function with \n, and then print that string so we can see it on stdout in the shell.

```
#!/usr/bin/env python
import click

import requests 
from collections.abc import Iterable 

def openalex_institution(query):
    'query is a list of terms in the query, or a string.'
    if isinstance(query, str):
        query = '+'.join(query.split())

    # We assume it is an iterable of strings.
    elif isinstance(query, Iterable):
        query = '+'.join(query)
        
    url = f'https://api.openalex.org/institutions?search={query}'
    req = requests.get(url)
    data = req.json()

    return [f'{result["display_name"]:50s}{result["works_count"]:10d}{result["cited_by_count"]:10d}'
            for result in data['results']]

@click.command(help='OpenAlex Institutions')
@click.argument('query', nargs=-1)
def main(query):
    print('\n'.join(openalex_institution(query)))
    
if __name__ == '__main__':
    main()
```    



In [23]:
! ./oa_inst2.py carnegie mellon university



Carnegie Mellon University                            107978   9708044
Carnegie Mellon University Qatar                         549     11205
Carnegie Mellon University Africa                         87       468
Carnegie Mellon University Australia                      27       188


Finally, we can import the function we wrote and use it in the notebook. This import works because the python file is in this directory. Later we will learn how to do this more generally. For now the critical idea is we have one file that can be used two different ways: one as a script in a shell, and one as a python library you can import the same function for use in a notebook, or even another script.



In [24]:
import oa_inst2

In [25]:
from oa_inst2 import openalex_institution
print('\n'.join(openalex_institution('carnegie+mellon+university')))



Carnegie Mellon University                            107978   9708044
Carnegie Mellon University Qatar                         549     11205
Carnegie Mellon University Africa                         87       468
Carnegie Mellon University Australia                      27       188


# Back to the author endpoint

You can access an author from a URL like this:
https://api.openalex.org/authors/https://orcid.org/0000-0003-2625-9232. Let's look at a few things. We get the name, number of works, and a url to those works. Although the url here says there are 172 works, it does not list them. Instead, it provides you with a url to get to them. Let's click on this url, and see what is there.



In [26]:
import requests

url = 'https://api.openalex.org/authors/https://orcid.org/0000-0003-2625-9232'
data = requests.get(url).json()
data['display_name'], data['works_count'], data['works_api_url']



('John R. Kitchin',
 172,
 'https://api.openalex.org/works?filter=author.id:A1760930984')

It is another set of json data.



In [27]:
works = requests.get(data['works_api_url']).json()
works['meta']



{'count': 172, 'db_response_time_ms': 77, 'page': 1, 'per_page': 25}

This new data has a new feature. There are 172 works, but on this "page" of data, there are only 25 results. We have to consider how to access all the pages to get the rest of the data. Paging is described here https://docs.openalex.org/how-to-use-the-api/get-lists-of-entities/paging. The gist is we have to add something to the url to increase the number of results per-page. We can increase it up to 200, and this example only has 172, so we do that.



In [28]:
works = requests.get(data['works_api_url'] + f'?page=1&per-page={data["works_count"]}').json()
works['meta']



{'count': 172, 'db_response_time_ms': 35, 'page': 1, 'per_page': 172}

Our goal now is to retrieve each work, and get the cited_by_count for each paper. Then, we will compute the H-index for this list of papers. The H-index is the number of papers that have at least H citations. The works are already sorted in descending citations here, so we don't have to sort them ourselves.



In [29]:
len(works['results'])

172

In [30]:
citations = [work['cited_by_count'] for work in works['results']]
for i, cite in enumerate(citations, start=1):
    if cite < i:
        print(f'H-index = {i - 1}')
        break
    print(f'{i:3d}{cite:8d}')



  1    6837
  2    3338
  3    2633
  4    1423
  5    1121
  6    1082
  7     387
  8     384
  9     274
 10     252
 11     237
 12     194
 13     162
 14     150
 15     123
 16      92
 17      90
 18      89
 19      86
 20      83
 21      81
 22      80
 23      76
 24      68
 25      65
 26      65
 27      64
 28      61
 29      59
 30      57
 31      56
 32      55
 33      54
 34      51
 35      47
 36      46
 37      44
 38      44
 39      42
 40      40
H-index = 40


# Group exercise

Work together to create a Python based shell script that takes an ORCID and computes the H-index for the author.



In [31]:
! cat orcid

#!/usr/bin/env python

import click
import requests

def hindex(orcid):
    
    url = f'https://api.openalex.org/authors/https://orcid.org/{orcid}'
    data = requests.get(url).json()

    works = (requests.get(data['works_api_url'] 
                          + f'?page=1&per-page={data["works_count"]}').json())
    
    citations = [work['cited_by_count'] for work in works['results']]
    for i, cite in enumerate(citations, start=1):
        if cite < i:
            return i - 1

@click.command(help='OpenAlex h-index')
@click.argument('orcid', nargs=1)
def main(orcid):
    print(f'h-index: {hindex(orcid)}')
    
if __name__ == '__main__':
    main()


In [32]:
! ./orcid 0000-0003-2625-9232

h-index: 40
